In [ ]:
import sys
sys.path.append("fwks_lib")

# limit the memory usage - this actually uses very small amount of memory
import keras.backend as K
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.75)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [ ]:
# generate denoising models

from fwks.tasks import Task, make_training_task
from fwks.model import DenoisingModel
from fwks import stage
from fwks.noise_gen import Static

def mk_model():
    return DenoisingModel([
        stage.Window(512, 256),
        stage.LogPowerFourier(),
        stage.RNN(width=257, depth=3),
        stage.L2Loss()
    ])


# this does not generate recordings for some reason

model = mk_model()
DenoisingTask = make_training_task(Static())
DenoisingTask.how_much = 80
DenoisingTask.mock = True


class MkDenoisingModel(metaclass=DenoisingTask):
    @classmethod
    def get_acoustic_model(self):
        return model


In [ ]:
MkDenoisingModel.run("/tmp/newcache")

In [ ]:
from fwks.tasks import AbstractModelTraining

denoising_model = MkDenoisingModel.get_model()


class BaselineMFCCModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return mfcc_model(denoising_model)


class HandcraftedModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return handcrafted_model(denoising_model)

    
class PLCModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return plc_model(denoising_model)

    
class MelFBModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return melfilterbank(denoising_model)

    
class CochleagramModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return cochleagram(denoising_model)
